In [1]:
# from bitarray import bitarray
from random import random, randrange, shuffle
from sortedcontainers import SortedSet
from collections import defaultdict

In [2]:
num_players = 5
starting_coins = 11
cards = list(range(3, 36))
discard = 9

In [3]:
num_players = 3
starting_coins = 4
cards = list(range(2, 16))
discard = 2

In [4]:
num_players = 2
starting_coins = 3
cards = list(range(2, 8))
discard = 1

In [5]:
"""We will store the game state as a bit array.
We need a bit for each card for each player to represent card posession,
enough bytes per player to represent their coins (limited by the total in play),
an integer to store the current card. Pot size and cards in deck is calculable from these."""

num_bits = int.bit_length(starting_coins * num_players) * num_players + len(cards) * num_players + int.bit_length(len(cards))
print(num_bits)

"""We could alternatively store ownership info of each card, i.e. an int
where 0 was unknown, 1 was in play, 2 etc. to indicate posesion by a player."""

num_bits = int.bit_length(starting_coins * num_players) * num_players + len(cards) * int.bit_length(num_players + 1)
print(num_bits)

# Upper bound on posible game states
# Real game states will be much fewer due to restrictions on coin movement
print('{:,.0f}'.format(2**num_bits))

21
18
262,144


In [6]:
class Strategy():
    def __init__(self):
        # Game tree will store at each node (default value):
        #   visits (0): number of times node has been visited
        #   [pass_regret, take_regret] ([0, 0]): cumulative regret
        #   strategy (0.5): probability of taking in this state
        #   average_strategy (0): running average of strategy over all visits
        self.data = defaultdict(self.default)
    def default(self):
        return {'visits': 0, 'regret': [0, 0],
                'strategy': 0.5, 'average_strategy': 0}
    def __getitem__(self, key):
        return self.data[key]
    def __len__(self):
        return len(self.data)
    def items(self):
        for key, val in self.data.items():
            yield key, val
class Player():
    def __init__(self, strategy):
        self.cards = SortedSet()
        self.coins = starting_coins
        self.history = {}
        self.memory = strategy
    def get_score(self):
        score = 0
        prev = 0 # (g) what if the first card is a 1? is there a 1 in this game?
        for card in self.cards:
            if card - prev > 1:  # not sequential
                score += card
            prev = card
        score -= self.coins
        return score
    def process_results(self, opponent_score, verbose=False):
        regret = self.get_score() - opponent_score  # high scores are bad, low are good
        for state, action in self.history.items():
            # Increment visits and record regret
            self.memory[state]['visits'] += 1
            if verbose: print('Visited this state {} times: {}'.format(self.memory[state]['visits'], state))
            self.memory[state]['regret'][action] += regret
            if verbose: print('Added {}  to {} for a total regret of {}'.format(regret, action, self.memory[state]['regret']))
            # Recalculate current strategy
            cum_regret = self.memory[state]['regret']
            cum_regret = [max(0, r) for r in cum_regret]
            if sum(cum_regret) > 0:
                self.memory[state]['strategy'] = cum_regret[False] / sum(cum_regret)
            else:
                self.memory[state]['strategy'] = 0.5
            # Update average strategy
            visits = self.memory[state]['visits']
            self.memory[state]['average_strategy'] *= (visits - 1) / visits
            self.memory[state]['average_strategy'] += self.memory[state]['strategy'] / visits
    def reset(self):
        self.history = {}
        self.cards = SortedSet()
        self.coins = starting_coins

class NoThanks():
    def __init__(self, players):
        self.players = players
        self.deck = list(cards)
        shuffle(self.deck)
        del self.deck[:discard]
        self.pot = 0
        self.current_player = randrange(num_players)
    def cur_card(self):
        return self.deck[0] if self.deck else None
    def deal(self):
        return self.deck.pop(0)
    def state(self):
        player_info = ((tuple(p.cards), p.coins) for p in self.players)
        return (self.cur_card(), self.pot, *player_info)
    def step(self, verbose=False, use_avg=False):
        player = self.players[self.current_player]
        state = game.state()
        strategy = player.memory[state]['average_strategy'] if use_avg else player.memory[state]['strategy']
        if player.coins == 0 or random() < strategy:  # take card
            player.history[state] = True
            if verbose: print('player {} ({:.2f}%, {:,.0f}) takes card {} and {} coins'.format(self.current_player, 100*strategy, player.memory[state]['visits'], self.cur_card(), self.pot))
            player.coins += self.pot
            self.pot = 0
            player.cards.add(self.deal())
        else:  # no thanks
            player.history[state] = False
            if verbose: print('player {} ({:.2f}%, {:,.0f}) refuses card {} and adds a coin to the pot ({})'.format(self.current_player, 100*strategy, player.memory[state]['visits'], self.cur_card(), self.pot + 1))
            player.coins -= 1
            self.pot += 1
        self.current_player = (self.current_player + 1) % len(self.players)
        if verbose: print(self.state())
    def play(self, verbose=False, use_avg=False):
        for player in self.players:
            player.reset()
        if verbose: print(self.state())
        while self.deck:
            self.step(verbose, use_avg)
        return [p.get_score() for p in self.players]
        
    def print_result(self):
        for idx, player in enumerate(self.players):
            print('Player {} ended with {} coins and the cards {} for a total of {} points.'.format(idx, player.coins, list(player.cards), player.get_score()))
        

In [7]:
strategy = Strategy()
players = [Player(strategy) for idx in range(num_players)]

In [8]:
# Naive players
game = NoThanks(players)
game.play(verbose=True)
game.print_result()

(3, 0, ((), 3), ((), 3))
player 1 (50.00%, 0) takes card 3 and 0 coins
(6, 0, ((), 3), ((3,), 3))
player 0 (50.00%, 0) takes card 6 and 0 coins
(7, 0, ((6,), 3), ((3,), 3))
player 1 (50.00%, 0) takes card 7 and 0 coins
(4, 0, ((6,), 3), ((3, 7), 3))
player 0 (50.00%, 0) refuses card 4 and adds a coin to the pot (1)
(4, 1, ((6,), 2), ((3, 7), 3))
player 1 (50.00%, 0) refuses card 4 and adds a coin to the pot (2)
(4, 2, ((6,), 2), ((3, 7), 2))
player 0 (50.00%, 0) takes card 4 and 2 coins
(5, 0, ((4, 6), 4), ((3, 7), 2))
player 1 (50.00%, 0) takes card 5 and 0 coins
(None, 0, ((4, 6), 4), ((3, 5, 7), 2))
Player 0 ended with 4 coins and the cards [4, 6] for a total of 6 points.
Player 1 ended with 2 coins and the cards [3, 5, 7] for a total of 13 points.


In [9]:
players[0].history

{(4, 0, ((6,), 3), ((3, 7), 3)): False,
 (4, 2, ((6,), 2), ((3, 7), 2)): True,
 (6, 0, ((), 3), ((3,), 3)): True}

In [10]:
scores = [player.get_score() for player in players]
total = sum(scores)
for player, score in zip(players, scores):
    mean_op_score = (total - score) / (num_players - 1)
    player.process_results(mean_op_score, verbose=True)
    print(' ')

Visited this state 1 times: (6, 0, ((), 3), ((3,), 3))
Added -7.0  to True for a total regret of [0, -7.0]
Visited this state 1 times: (4, 0, ((6,), 3), ((3, 7), 3))
Added -7.0  to False for a total regret of [-7.0, 0]
Visited this state 1 times: (4, 2, ((6,), 2), ((3, 7), 2))
Added -7.0  to True for a total regret of [0, -7.0]
 
Visited this state 1 times: (3, 0, ((), 3), ((), 3))
Added 7.0  to True for a total regret of [0, 7.0]
Visited this state 1 times: (7, 0, ((6,), 3), ((3,), 3))
Added 7.0  to True for a total regret of [0, 7.0]
Visited this state 1 times: (4, 1, ((6,), 2), ((3, 7), 3))
Added 7.0  to False for a total regret of [7.0, 0]
Visited this state 1 times: (5, 0, ((4, 6), 4), ((3, 7), 2))
Added 7.0  to True for a total regret of [0, 7.0]
 


In [11]:
scores, total, mean_op_score

([6, 13], 19, 6.0)

In [12]:
%%time
for idx in range(100000):
    game = NoThanks(players)
    scores = game.play()
    total = sum(scores)
    for player, score in zip(players, scores):
        mean_op_score = (total - score) / (num_players - 1)
        player.process_results(mean_op_score)

print('Visits {:,.0f} states in {:,.0f} turns'.format(len(strategy), sum(x['visits'] for k, x in strategy.items())))

Visits 16,372 states in 861,251 turns
CPU times: user 13.4 s, sys: 55.5 ms, total: 13.5 s
Wall time: 13.6 s


In [13]:
# Trained players
game = NoThanks(players)
game.play(verbose=True, use_avg=True)
game.print_result()

(2, 0, ((), 3), ((), 3))
player 1 (64.35%, 16,450) takes card 2 and 0 coins
(5, 0, ((), 3), ((2,), 3))
player 0 (100.00%, 1,073) takes card 5 and 0 coins
(4, 0, ((5,), 3), ((2,), 3))
player 1 (70.28%, 498) takes card 4 and 0 coins
(7, 0, ((5,), 3), ((2, 4), 3))
player 0 (1.06%, 162) refuses card 7 and adds a coin to the pot (1)
(7, 1, ((5,), 2), ((2, 4), 3))
player 1 (40.55%, 164) takes card 7 and 1 coins
(6, 0, ((5,), 2), ((2, 4, 7), 4))
player 0 (50.00%, 63) takes card 6 and 0 coins
(None, 0, ((5, 6), 2), ((2, 4, 7), 4))
Player 0 ended with 2 coins and the cards [5, 6] for a total of 3 points.
Player 1 ended with 4 coins and the cards [2, 4, 7] for a total of 9 points.
